In [1]:
import pandas as pd
import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, Dense, GlobalMaxPooling1D

Using TensorFlow backend.


In [2]:
train_data = pd.read_csv('data/train.csv')
val_data = pd.read_csv('data/validation.csv')
print(len(train_data), len(val_data))

105000 15000


In [3]:
train_data.head()

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...",-2,-2,-2,-2,1,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,1,-2,1,-2
1,1,"""第三次参加大众点评网霸王餐的活动。这家店给人整体感觉一般。首先环境只能算中等，其次霸王餐提...",-2,-2,-2,-2,-2,-2,-2,0,...,0,0,0,0,1,-2,-2,-2,1,-2
2,2,"""4人同行 点了10个小吃\n榴莲酥 榴莲味道不足 松软 奶味浓\n虾饺 好吃 两颗大虾仁\...",-2,-2,-2,-2,0,-2,1,0,...,-2,-2,1,-2,0,1,-2,-2,0,-2
3,3,"""之前评价了莫名其妙被删 果断继续差评！ 换了菜单 价格更低 开始砸牌子 但套餐还是有150...",-2,-2,-2,-2,-2,-2,-2,0,...,-2,-2,-2,-2,-2,-1,-2,-2,-1,-1
4,4,"""出乎意料地惊艳，椰子鸡清热降火，美容养颜，大大满足了爱吃火锅怕上火星人。椰子冻是帅帅的老板...",-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,1,1,-2,1,-2


In [4]:
train_data.columns[2:]

Index(['location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again'],
      dtype='object')

- 位置：3个，location_traffic_convenience, location_distance_from_business_district, location_easy_to_find
- 服务：4个，service_wait_time, service_waiters_attitude, service_parking_convenience, service_serving_speed
- 价格：3个，price_level, price_cost_effective, price_discount
- 环境：4个，environment_decoration, environment_noise, environment_space, environment_cleaness
- 菜品：4个，dish_portion, dish_taste, dish_look, dish_recommendation
- 其他：2个，others_overall_experience, others_willing_to_consume_again

In [5]:
x_train = list(map(lambda x: x[1:-1], train_data['content']))
x_val = list(map(lambda x: x[1:-1], val_data['content']))

In [6]:
tokenizer = Tokenizer(num_words=10000, char_level=True)
tokenizer.fit_on_texts(x_train)

In [7]:
sequences = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(sequences, maxlen=1000)

sequences = tokenizer.texts_to_sequences(x_val)
x_val = pad_sequences(sequences, maxlen=1000)

In [13]:
test_data = pd.read_csv('data/test.csv')
test_data.head()

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""我想说他们家的优惠活动好持久啊，我预售的时候买的券，前两天心血来潮去吃的活动还在继续\n首...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"""终于开到心心念念的LAB loft。第一次来就随便点也一些～【香辣虾意面】蛮辣的，但其实一...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"""地理位置好，交通方便，就在124车站对面交通方便，很好，我晚上7点多去买的了，已经没有什么...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"""运气很好，抽中了大众点评的霸王餐。这家主题餐厅心仪已久了，种种原因一直未能成行，没想到抽中...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"""幸运随点评团体验霸王餐，心情好~蜀九香刚进驻泉州不久，招牌大名气响，以至于刚到店门口的我被...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
x_test = list(map(lambda x: x[1:-1], test_data['content']))
sequences = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(sequences, maxlen=1000)

In [8]:
y_train_ltc = pd.get_dummies(train_data['location_traffic_convenience'])[[-2, -1, 0, 1]].values
y_train_ldfbd = pd.get_dummies(train_data['location_distance_from_business_district'])[[-2, -1, 0, 1]].values
y_train_letf = pd.get_dummies(train_data['location_easy_to_find'])[[-2, -1, 0, 1]].values

y_train_swt = pd.get_dummies(train_data['service_wait_time'])[[-2, -1, 0, 1]].values
y_train_swa = pd.get_dummies(train_data['service_waiters_attitude'])[[-2, -1, 0, 1]].values
y_train_spc = pd.get_dummies(train_data['service_parking_convenience'])[[-2, -1, 0, 1]].values
y_train_sss = pd.get_dummies(train_data['service_serving_speed'])[[-2, -1, 0, 1]].values

y_train_pl = pd.get_dummies(train_data['price_level'])[[-2, -1, 0, 1]].values
y_train_pce = pd.get_dummies(train_data['price_cost_effective'])[[-2, -1, 0, 1]].values
y_train_pd = pd.get_dummies(train_data['price_discount'])[[-2, -1, 0, 1]].values

y_train_ed = pd.get_dummies(train_data['environment_decoration'])[[-2, -1, 0, 1]].values
y_train_en = pd.get_dummies(train_data['environment_noise'])[[-2, -1, 0, 1]].values
y_train_es = pd.get_dummies(train_data['environment_space'])[[-2, -1, 0, 1]].values
y_train_ec = pd.get_dummies(train_data['environment_cleaness'])[[-2, -1, 0, 1]].values

y_train_dp = pd.get_dummies(train_data['dish_portion'])[[-2, -1, 0, 1]].values
y_train_dt = pd.get_dummies(train_data['dish_taste'])[[-2, -1, 0, 1]].values
y_train_dl = pd.get_dummies(train_data['dish_look'])[[-2, -1, 0, 1]].values
y_train_dr = pd.get_dummies(train_data['dish_recommendation'])[[-2, -1, 0, 1]].values

y_train_ooe = pd.get_dummies(train_data['others_overall_experience'])[[-2, -1, 0, 1]].values
y_train_owtca = pd.get_dummies(train_data['others_willing_to_consume_again'])[[-2, -1, 0, 1]].values

In [9]:
y_val_ltc = pd.get_dummies(val_data['location_traffic_convenience'])[[-2, -1, 0, 1]].values
y_val_ldfbd = pd.get_dummies(val_data['location_distance_from_business_district'])[[-2, -1, 0, 1]].values
y_val_letf = pd.get_dummies(val_data['location_easy_to_find'])[[-2, -1, 0, 1]].values

y_val_swt = pd.get_dummies(val_data['service_wait_time'])[[-2, -1, 0, 1]].values
y_val_swa = pd.get_dummies(val_data['service_waiters_attitude'])[[-2, -1, 0, 1]].values
y_val_spc = pd.get_dummies(val_data['service_parking_convenience'])[[-2, -1, 0, 1]].values
y_val_sss = pd.get_dummies(val_data['service_serving_speed'])[[-2, -1, 0, 1]].values

y_val_pl = pd.get_dummies(val_data['price_level'])[[-2, -1, 0, 1]].values
y_val_pce = pd.get_dummies(val_data['price_cost_effective'])[[-2, -1, 0, 1]].values
y_val_pd = pd.get_dummies(val_data['price_discount'])[[-2, -1, 0, 1]].values

y_val_ed = pd.get_dummies(val_data['environment_decoration'])[[-2, -1, 0, 1]].values
y_val_en = pd.get_dummies(val_data['environment_noise'])[[-2, -1, 0, 1]].values
y_val_es = pd.get_dummies(val_data['environment_space'])[[-2, -1, 0, 1]].values
y_val_ec = pd.get_dummies(val_data['environment_cleaness'])[[-2, -1, 0, 1]].values

y_val_dp = pd.get_dummies(val_data['dish_portion'])[[-2, -1, 0, 1]].values
y_val_dt = pd.get_dummies(val_data['dish_taste'])[[-2, -1, 0, 1]].values
y_val_dl = pd.get_dummies(val_data['dish_look'])[[-2, -1, 0, 1]].values
y_val_dr = pd.get_dummies(val_data['dish_recommendation'])[[-2, -1, 0, 1]].values

y_val_ooe = pd.get_dummies(val_data['others_overall_experience'])[[-2, -1, 0, 1]].values
y_val_owtca = pd.get_dummies(val_data['others_willing_to_consume_again'])[[-2, -1, 0, 1]].values

In [10]:
def build_model():
    model = Sequential()
    model.add(Embedding(10000, 128, input_length=1000))
    model.add(Conv1D(32, 5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
    return model

#### 位置

In [11]:
model1 = build_model()
model1.fit(x_train, y_train_ltc, epochs=5, batch_size=64, validation_data=(x_val, y_val_ltc))

Train on 105000 samples, validate on 15000 samples
Epoch 1/5
105000/105000 [==============================] - 34s 322us/step - loss: 0.2615 - acc: 0.9267 - val_loss: 0.2141 - val_acc: 0.9394
Epoch 2/5
105000/105000 [==============================] - 26s 243us/step - loss: 0.2097 - acc: 0.9389 - val_loss: 0.1945 - val_acc: 0.9409
Epoch 3/5
105000/105000 [==============================] - 26s 250us/step - loss: 0.1952 - acc: 0.9422 - val_loss: 0.1966 - val_acc: 0.9434
Epoch 4/5
105000/105000 [==============================] - 26s 244us/step - loss: 0.1868 - acc: 0.9443 - val_loss: 0.2004 - val_acc: 0.9408
Epoch 5/5
105000/105000 [==============================] - 27s 260us/step - loss: 0.1792 - acc: 0.9464 - val_loss: 0.2084 - val_acc: 0.9414


In [19]:
labels = [-2, -1, 0, 1]
ltc_preds = model1.predict_classes(x_test)
ltc_preds = [labels[i] for i in ltc_preds]

In [20]:
model1 = build_model()
model1.fit(x_train, y_train_ldfbd, epochs=2, batch_size=64, validation_data=(x_val, y_val_ldfbd))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 245us/step - loss: 0.3535 - acc: 0.8751 - val_loss: 0.3238 - val_acc: 0.8821
Epoch 2/2
105000/105000 [==============================] - 25s 243us/step - loss: 0.3118 - acc: 0.8859 - val_loss: 0.3217 - val_acc: 0.8793


In [21]:
labels = [-2, -1, 0, 1]
ldfbd_preds = model1.predict_classes(x_test)
ldfbd_preds = [labels[i] for i in ldfbd_preds]

In [22]:
model1 = build_model()
model1.fit(x_train, y_train_letf, epochs=2, batch_size=64, validation_data=(x_val, y_val_letf))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 245us/step - loss: 0.3598 - acc: 0.8914 - val_loss: 0.3069 - val_acc: 0.9061
Epoch 2/2
105000/105000 [==============================] - 25s 241us/step - loss: 0.2977 - acc: 0.9079 - val_loss: 0.3018 - val_acc: 0.9083


In [23]:
labels = [-2, -1, 0, 1]
letf_preds = model1.predict_classes(x_test)
letf_preds = [labels[i] for i in letf_preds]

#### 服务

In [24]:
model1 = build_model()
model1.fit(x_train, y_train_swt, epochs=2, batch_size=64, validation_data=(x_val, y_val_swt))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 246us/step - loss: 0.3323 - acc: 0.8931 - val_loss: 0.2944 - val_acc: 0.9057
Epoch 2/2
105000/105000 [==============================] - 25s 242us/step - loss: 0.2876 - acc: 0.9062 - val_loss: 0.3058 - val_acc: 0.9017


In [25]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
swt_preds = [labels[i] for i in preds]

In [26]:
model1 = build_model()
model1.fit(x_train, y_train_swa, epochs=2, batch_size=64, validation_data=(x_val, y_val_swa))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 247us/step - loss: 0.5708 - acc: 0.7878 - val_loss: 0.4964 - val_acc: 0.8132
Epoch 2/2
105000/105000 [==============================] - 25s 242us/step - loss: 0.4569 - acc: 0.8300 - val_loss: 0.4731 - val_acc: 0.8196


In [27]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
swa_preds = [labels[i] for i in preds]

In [28]:
model1 = build_model()
model1.fit(x_train, y_train_spc, epochs=2, batch_size=64, validation_data=(x_val, y_val_spc))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 246us/step - loss: 0.1247 - acc: 0.9629 - val_loss: 0.0876 - val_acc: 0.9707
Epoch 2/2
105000/105000 [==============================] - 25s 241us/step - loss: 0.0851 - acc: 0.9727 - val_loss: 0.0903 - val_acc: 0.9707


In [29]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
spc_preds = [labels[i] for i in preds]

In [30]:
model1 = build_model()
model1.fit(x_train, y_train_sss, epochs=2, batch_size=64, validation_data=(x_val, y_val_sss))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 247us/step - loss: 0.2997 - acc: 0.9101 - val_loss: 0.2573 - val_acc: 0.9216
Epoch 2/2
105000/105000 [==============================] - 25s 241us/step - loss: 0.2350 - acc: 0.9264 - val_loss: 0.2405 - val_acc: 0.9269


In [31]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
sss_preds = [labels[i] for i in preds]

#### 价格

In [32]:
model1 = build_model()
model1.fit(x_train, y_train_pl, epochs=2, batch_size=64, validation_data=(x_val, y_val_pl))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 244us/step - loss: 0.6514 - acc: 0.7432 - val_loss: 0.5768 - val_acc: 0.7771
Epoch 2/2
105000/105000 [==============================] - 26s 243us/step - loss: 0.5403 - acc: 0.7919 - val_loss: 0.5592 - val_acc: 0.7821


In [33]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
pl_preds = [labels[i] for i in preds]

In [34]:
model1 = build_model()
model1.fit(x_train, y_train_pce, epochs=2, batch_size=64, validation_data=(x_val, y_val_pce))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 250us/step - loss: 0.3958 - acc: 0.8649 - val_loss: 0.3457 - val_acc: 0.8735
Epoch 2/2
105000/105000 [==============================] - 26s 243us/step - loss: 0.3310 - acc: 0.8834 - val_loss: 0.3294 - val_acc: 0.8837


In [35]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
pce_preds = [labels[i] for i in preds]

In [36]:
model1 = build_model()
model1.fit(x_train, y_train_pd, epochs=2, batch_size=64, validation_data=(x_val, y_val_pd))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 249us/step - loss: 0.5104 - acc: 0.8054 - val_loss: 0.4554 - val_acc: 0.8282
Epoch 2/2
105000/105000 [==============================] - 25s 242us/step - loss: 0.4420 - acc: 0.8315 - val_loss: 0.4486 - val_acc: 0.8294


In [37]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
pd_preds = [labels[i] for i in preds]

#### 环境

In [38]:
model1 = build_model()
model1.fit(x_train, y_train_ed, epochs=2, batch_size=64, validation_data=(x_val, y_val_ed))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 249us/step - loss: 0.5468 - acc: 0.8086 - val_loss: 0.4593 - val_acc: 0.8387
Epoch 2/2
105000/105000 [==============================] - 25s 242us/step - loss: 0.4516 - acc: 0.8408 - val_loss: 0.4508 - val_acc: 0.8380


In [39]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
ed_preds = [labels[i] for i in preds]

In [40]:
model1 = build_model()
model1.fit(x_train, y_train_en, epochs=2, batch_size=64, validation_data=(x_val, y_val_en))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 250us/step - loss: 0.4910 - acc: 0.8301 - val_loss: 0.4202 - val_acc: 0.8541
Epoch 2/2
105000/105000 [==============================] - 25s 243us/step - loss: 0.4022 - acc: 0.8585 - val_loss: 0.4050 - val_acc: 0.8571


In [41]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
en_preds = [labels[i] for i in preds]

In [42]:
model1 = build_model()
model1.fit(x_train, y_train_es, epochs=2, batch_size=64, validation_data=(x_val, y_val_es))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 250us/step - loss: 0.6334 - acc: 0.7712 - val_loss: 0.5370 - val_acc: 0.8030
Epoch 2/2
105000/105000 [==============================] - 25s 241us/step - loss: 0.5121 - acc: 0.8134 - val_loss: 0.5189 - val_acc: 0.8105


In [43]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
es_preds = [labels[i] for i in preds]

In [44]:
model1 = build_model()
model1.fit(x_train, y_train_ec, epochs=2, batch_size=64, validation_data=(x_val, y_val_ec))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 249us/step - loss: 0.5247 - acc: 0.8220 - val_loss: 0.4529 - val_acc: 0.8462
Epoch 2/2
105000/105000 [==============================] - 25s 242us/step - loss: 0.4347 - acc: 0.8528 - val_loss: 0.4222 - val_acc: 0.8558


In [45]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
ec_preds = [labels[i] for i in preds]

#### 菜品

In [46]:
model1 = build_model()
model1.fit(x_train, y_train_dp, epochs=2, batch_size=64, validation_data=(x_val, y_val_dp))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 249us/step - loss: 0.7340 - acc: 0.7278 - val_loss: 0.6424 - val_acc: 0.7619
Epoch 2/2
105000/105000 [==============================] - 25s 242us/step - loss: 0.6171 - acc: 0.7732 - val_loss: 0.6755 - val_acc: 0.7515


In [47]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
dp_preds = [labels[i] for i in preds]

In [48]:
model1 = build_model()
model1.fit(x_train, y_train_dt, epochs=2, batch_size=64, validation_data=(x_val, y_val_dt))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 251us/step - loss: 0.6739 - acc: 0.7104 - val_loss: 0.6041 - val_acc: 0.7420
Epoch 2/2
105000/105000 [==============================] - 25s 243us/step - loss: 0.5679 - acc: 0.7589 - val_loss: 0.5851 - val_acc: 0.7462


In [49]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
dt_preds = [labels[i] for i in preds]

In [50]:
model1 = build_model()
model1.fit(x_train, y_train_dl, epochs=2, batch_size=64, validation_data=(x_val, y_val_dl))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 251us/step - loss: 0.6489 - acc: 0.7791 - val_loss: 0.6117 - val_acc: 0.7886
Epoch 2/2
105000/105000 [==============================] - 26s 243us/step - loss: 0.5765 - acc: 0.7951 - val_loss: 0.5913 - val_acc: 0.7913


In [51]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
dl_preds = [labels[i] for i in preds]

In [52]:
model1 = build_model()
model1.fit(x_train, y_train_dr, epochs=2, batch_size=64, validation_data=(x_val, y_val_dr))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 251us/step - loss: 0.3759 - acc: 0.8764 - val_loss: 0.3583 - val_acc: 0.8755
Epoch 2/2
105000/105000 [==============================] - 26s 243us/step - loss: 0.3222 - acc: 0.8942 - val_loss: 0.3332 - val_acc: 0.8895


In [53]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
dr_preds = [labels[i] for i in preds]

#### 其他

In [54]:
model1 = build_model()
model1.fit(x_train, y_train_ooe, epochs=2, batch_size=64, validation_data=(x_val, y_val_ooe))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 252us/step - loss: 0.6235 - acc: 0.7553 - val_loss: 0.5620 - val_acc: 0.7771
Epoch 2/2
105000/105000 [==============================] - 26s 244us/step - loss: 0.5353 - acc: 0.7932 - val_loss: 0.5488 - val_acc: 0.7849


In [55]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
ooe_preds = [labels[i] for i in preds]

In [56]:
model1 = build_model()
model1.fit(x_train, y_train_owtca, epochs=2, batch_size=64, validation_data=(x_val, y_val_owtca))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 27s 253us/step - loss: 0.5447 - acc: 0.7997 - val_loss: 0.4794 - val_acc: 0.8219
Epoch 2/2
105000/105000 [==============================] - 26s 244us/step - loss: 0.4508 - acc: 0.8359 - val_loss: 0.4538 - val_acc: 0.8366


In [57]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
owtca_preds = [labels[i] for i in preds]

#### 测试

- 位置：3个，location_traffic_convenience, location_distance_from_business_district, location_easy_to_find
- 服务：4个，service_wait_time, service_waiters_attitude, service_parking_convenience, service_serving_speed
- 价格：3个，price_level, price_cost_effective, price_discount
- 环境：4个，environment_decoration, environment_noise, environment_space, environment_cleaness
- 菜品：4个，dish_portion, dish_taste, dish_look, dish_recommendation
- 其他：2个，others_overall_experience, others_willing_to_consume_again

In [59]:
test_data['location_traffic_convenience'] = ltc_preds
test_data['location_distance_from_business_district'] = ldfbd_preds
test_data['location_easy_to_find'] = letf_preds

test_data['service_wait_time'] = swt_preds
test_data['service_waiters_attitude'] = swa_preds
test_data['service_parking_convenience'] = spc_preds
test_data['service_serving_speed'] = sss_preds

test_data['price_level'] = pl_preds
test_data['price_cost_effective'] = pce_preds
test_data['price_discount'] = pd_preds

test_data['environment_decoration'] = ed_preds
test_data['environment_noise'] = en_preds
test_data['environment_space'] = es_preds
test_data['environment_cleaness'] = ec_preds

test_data['dish_portion'] = dp_preds
test_data['dish_taste'] = dt_preds
test_data['dish_look'] = dl_preds
test_data['dish_recommendation'] = dr_preds

test_data['others_overall_experience'] = ooe_preds
test_data['others_willing_to_consume_again'] = owtca_preds

In [60]:
test_data

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""我想说他们家的优惠活动好持久啊，我预售的时候买的券，前两天心血来潮去吃的活动还在继续\n首...",-2,-2,-2,-2,1,-2,-2,0,...,0,-2,-2,1,1,1,-2,-2,1,1
1,1,"""终于开到心心念念的LAB loft。第一次来就随便点也一些～【香辣虾意面】蛮辣的，但其实一...",-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-1,0,-2,-2,0,-2
2,2,"""地理位置好，交通方便，就在124车站对面交通方便，很好，我晚上7点多去买的了，已经没有什么...",1,1,1,-2,0,-2,-2,0,...,-2,-2,-2,-2,-2,1,-2,-2,1,-1
3,3,"""运气很好，抽中了大众点评的霸王餐。这家主题餐厅心仪已久了，种种原因一直未能成行，没想到抽中...",1,1,-2,-2,1,1,-2,-2,...,1,-2,-2,-2,-2,1,-2,1,1,1
4,4,"""幸运随点评团体验霸王餐，心情好~蜀九香刚进驻泉州不久，招牌大名气响，以至于刚到店门口的我被...",-2,-2,-2,0,1,-2,-2,-2,...,1,-1,-2,1,-2,1,-2,-2,1,-2
5,5,"""尽管韩国烤肉店在无锡已经有很多家了，但因为味道好吃再加上在无锡很有人气，依旧挡不住新店的开...",-1,-2,1,-2,1,-2,-2,-1,...,0,-2,-2,-2,-1,1,-2,1,1,-2
6,6,"""店铺在乙烯生活二区的西北角，旁边是国旅的旗舰店，门口就是红绿灯，挺好找的。因为这是两磅一的...",-2,-2,1,-2,1,-2,-2,1,...,-2,-2,-2,-2,1,1,-2,-2,1,-2
7,7,"""朋友聚会来滴，这个地方真心不错哇，又能撸串，又能唱歌，还能看现场歌手演唱。最主要是可以上台...",-2,-2,-2,-2,-2,-2,-2,-2,...,-2,1,-2,-2,-2,1,-2,-2,1,1
8,8,"""超喜欢这家的面食，经常来吃。这家风格及产品都很像京城御面，有时会怀疑是不是同一家的。面条比...",-2,-2,-2,-2,-2,-2,-2,1,...,-2,-2,-2,-2,1,1,-2,-2,1,-2
9,9,"""广西阳朔，前一天晚上吃的特色啤酒鱼，就看到了隔壁有家螺蛳粉，第二天过来吃。个人比较喜欢粉，...",-2,-2,1,-2,-2,1,1,0,...,-2,-2,1,1,-2,0,-2,-2,0,-2


In [61]:
test_data.to_csv('data/submission.csv', index=False)